# Aritmética de ponto flutuante em computadores

## $ \S 1 $ Introdução

O principal objetivo dos métodos numéricos é desenvolver algoritmos e técnicas
eficazes para resolver problemas matemáticos e, no processo, ajudar-nos a obter
um maior entendimento sobre eles. Os métodos numéricos são especialmente úteis
quando soluções analíticas ou em forma fechada não estão disponíveis ou são
muito difíceis de se calcular.  Números são a base de toda computação. Portanto
é necessário que primeiro compreendamos como eles são representados em uma
máquina, como a capacidade de memória e poder de processamento finitos
inevitavelmente levam a erros, e como estes erros podem ser controlados.

## $ \S 2 $ Números inteiros

Se usarmos $ 32 $ bits para armazenar diretamente os dígitos de
__números inteiros__, o menor e maior valor que podem ser representados na
máquina são
$$
-2^{31} = -2,147,483,648 \quad \text{e} \quad  2^{31} - 1 = 2,147,483,647\,,
$$
ou seja, entre $ \pm 2 $ bilhões, aproximadamente. Note que estes números
são relativamente pequenos. Esta é de fato a representação padrão para inteiros
na maioria das linguages de programação, como Java ou C (mas não em Python).

## $ \S 3 $ O sistema de representação de números de ponto fixo

No sistema dos __números de ponto fixo__ para representação de números reais,
especifica-se uma base $ b \ge 2 $, digamos a decimal, e uma quantidade fixa de
dígitos para se representar a parte inteira e a parte fracionária, digamos $ 3 $
e $ 4 $ respectivamente. Poderíamos portanto representar os números
\begin{align*}
\pi& \quad \text{como}& 3&.1416 \\
-\frac{\pi}{1000}& \quad \text{como}& -0&.0031 \\
123 & \quad \text{como}& 123&.0 \\
-123.456789& \quad \text{como}& -123&.4568
\end{align*}
Observe que neste sistema a diferença entre dois números sucessivos é sempre a
mesma ($ 0.0001 $ para as escolhas que fizemos).

Como veremos, o sistema de ponto fixo é muito inflexível e ineficiente para ser
utilizado na prática. Usando base $ b = 2 $ e $ 32 $ bits para armazenar cada
número positivo, com $ 16 $ bits reservados para a parte inteira e $ 16 $ para a
parte fracionária, só seria possível representar números entre $ 2^{-16} \approx
0,00002 $ e $ 2^{16} = 65536 $.


## $ \S 4 $ O sistema de representação de números de ponto flutuante

### $ 4.1 $ Definição dos números de ponto flutuante

O sistema de ponto flutuante é um pilar dos métodos numéricos, fornecendo um
meio eficiente de representar e manipular números reais em hardware e software.
Os números de ponto flutuante são números expressos num formato específico, que
nos permite representar tanto magnitudes grandes quanto pequenas, mantendo
precisão razoável. O padrão [IEEE 754](https://en.wikipedia.org/wiki/IEEE_754)
(estabelecido em 1985), que define os formatos de números de ponto flutuante
utilizados comumente, tornou-se parte integrante da engenharia dos sistemas
modernos de computação e dos algoritmos numéricos. Em particular, ele também é
seguido em Python.

Um número de ponto flutuante consiste do produto de duas componentes: uma
__mantissa__ (ou _significando_) e uma __base__ elevada a um __expoente__, por
exemplo:
$$
-1234.56 = -1.23456
\times 10^3 \,.
$$
Mais geralmente, um __número de ponto flutante__ tem a forma
$$
x = \text{(sinal)}\, \, \text{mantissa}\, \times \,\text{base}^{\text{expoente}}\,.
$$
_Devem ser respeitadas as seguintes restrições sobre estes elementos:_
* O __sinal__ $ \pm $ é representado por um único bit.
* A __base__ $ b $ pode ser qualquer inteiro $ \ge 2 $ (ou seja), mas esta escolha é fixa.
  Na maioria esmagadora dos casos, é utilizado o sistema binário ($ b = 2 $).
* O __expoente__ é armazenado utilizando-se uma quantidade fixa de bits, por exemplo
  $ 8 $ para precisão simples. Dentre as $ 2^8 = 256 $
  possibilidades, duas sempre são reservadas para denotar valores especiais,
  como infinito (`inf` em Python) ou _NaN_ (_Not a Number_, `NaN`). Sendo assim,
  no caso da precisão simples, os possíveis expoentes são aqueles entre $ -126 $
  e $ +127 $. Esta restrição limita as _magnitudes_ dos números que podem ser
  representados precisamente pela máquina. Se o expoente de um número não está
  dentro desta faixa, o resultado pode ser um erro de __overflow__ (muito grande
  para ser representado) ou de __underflow__ (muito pequeno para ser
  representado).
* A __mantissa__ ou __significando__ deve estar no intervalo $ [1, b) $,
  onde $ b $ é a base. Sendo assim, neste formato a mantissa de um número
  envolve exatamente um dígito antes do ponto (e este dígito não pode ser nulo).
  Por exemplo, quando $ b = 10 $ é a base, $ 4.321 $ é um mantissa válida, mas $
  0.4321 $ e $ 43.21 $ não são. A mantissa é representada no sistema de precisão
  simples através de $ 23 $ bits. Na verdade quando a base é $ 2 $, libera-se um
  bit adicional para armazenagem, já que como a mantissa deve estar em $ [1, 2)
  $, o dígito antes do ponto só pode ser $ 1 $; este bit é chamado de _bit
  implícito_ ou _escondido_.  Seja como for, a limitação do número de bits
  reservados para a mantissa impõe uma restrição à _precisão_ com que um número
  pode ser representado.
  
No caso do formato de __precisão simples__ são utilizados portanto
$$
32 \text{ bits} = 1 \text{ bit para o sinal} +
8 \text{ bits para o expoente} + 23\text{ bits para a mantissa}
$$
para se representar um número de ponto flutuante. Observe que mesmo com a
utilização de apenas $ 32 $ bits, através deste formato é possível representar
números entre $ 2^{-126} $ e $ 2^{127} $ aproximadamente, uma faixa muito mais
ampla que aquela representável usando-se o formato de ponto fixo.

📝 O padrão IEEE 754 também contém especificações para representação de números
com dupla ($ 64 $ bits), quádrupla ($ 128 $ bits) e meia ($ 16 $ bits) precisão,
dentre outros.

📝 O termo "ponto flutuante" se refere ao fato que a posição do ponto de base
não é fixa. À medida que o expoente muda, o ponto pode "flutuar" entre
diferentes posições, permitindo que a representação cubra eficientemente uma
ampla gama de números. Python representa os números de ponto flutuante com o
tipo `float`; a sua representação mais fiel é aquela da chamada
_notação científica_: `<mantissa>e<expoente>` (em que a base $ b = 10 $). Por
exemplo $ 0.012345 $ corresponde nesta notação a `1.2345e-2`.


In [1]:
1.2345e-2

0.012345

### $ 4.2 $ Características do sistema de números de ponto flutuante

<div class="alert alert-info">  Para tornar a discussão mais simples e concreta,
de agora em diante assumiremos que no nosso sistema de números de ponto
flutuante:
    <ul>
        <li> A base $ b $ é $ 10 $.
        <li> As mantissas são constituídas de $ 3 $ dígitos.
        <li> Os expoentes variam entre $ -9 $ e $ 9 $.
        </li></ul></div>
</div>

Neste caso:
* O zero é representado por $$ 0 = 0.00 \times 10^{-9}\,. $$ Este é o único
  número cuja mantissa não respeita as restrições descritas na $ \S 4.1 $.
  Para manter a unicidade e para conseguirmos comparar números 
  de maneira consistente, o zero _não_ é representado na forma 
  $ 0.00 \times 10^{e} $ para qualquer expoente $ e \ne -9 $.
* O maior número representável neste sistema é $ 9.99 \times 10^9 $
  (aproximadamente $ 10 $ bilhões), e o menor número positivo é
  $ 1.00 \times 10^{-9} $.
* Chamamos de __década__ o conjunto de números que têm um mesmo expoente $ e $.
  Em cada década do nosso sistema, podem ser representados $ 900 $ números:
  aqueles com mantissa entre $ 1.00 $ e $ 9.99 $.
* Dentro da década de expoente $ e $, números sucessivos estão igualmente
  espaçados (por $ 10^{e - 2} $, no nosso formato). Em particular, esta
  distância aumenta exponencialmente conforme $ e $ aumenta.
* O padrão IEEE 754 detalha vários modos de se aproximar um número que
  não pode ser representado exatamente neste formato, de acordo com 
  a necessidade da aplicação sob consideração. Por exemplo, para
  representar $ 2.345 $ neste sistema, podem-se utilizar:
    * __Truncamento__, significando que os dígitos excedentes são simplesmente
      descartados; neste caso o truncamento seria portanto
      $ 2.34 \times 10^0 $.
    * __Arredondamento, desempate por cima__, significando que o número é
    substituído por aquele mais próximo dentro do formato e, em caso de empate
    entre duas opções, pela maior delas; no exemplo, $ 2.35 \times 10^0 $.
    * __Arredondamento, desempate por dígito menos significativo par__
    __(modo default)__, significando
    que o número é substituído pelo mais próximo dentro do formato e que,
    no caso de haver duas opções, é escolhido aquele cujo dígito menos 
    significativo é par; no caso, $ 2.34 \times 10^0 $. A vantagem deste modo é
    que em casos ambíguos o arredondamento é feito sem qualquer viés, já
    que metade dos dígitos em base $ 10 $ (ou base $ 2 $) é par e a outra metade é
    ímpar. Se em casos de empate o arredondamento sempre é feito por cima, por
    exemplo, em algumas situações isto pode gerar um acúmulo de erros de
    aproximação, por todos terem a mesma direção.
* Os resultados _intermediários_ de operações com números de ponto-flutuante
  são armazenados com precisão extra, em geral precisão dupla. Nos nossos
  exemplos, permitiremos que a mantissa destes números auxiliares seja
  representada com $ 6 $ dígitos (em vez de $ 3 $). Além disto, relaxamos
  a exigência que a mantissa contenha exatamente um dígito não-nulo antes do ponto.
  Contudo, ao final da operação sempre é necessário trazer o resultado de volta
  ao formato especificado, por algum dos três tipos de aproximação considerados
  no ponto acima.

__Exercício:__ Mostre que o nosso sistema consiste de exatamente $ 34\,201 $
números diferentes.

A precisão finita inerente à representação de números reais utilizando um
número limitado de dígitos não apenas restringe a classe de números que
podem ser representados de maneira exata, mas também traz inevitavelmente a
possibilidade de erros mesmo nas operações aritméticas mais simples.
Estas limitações exigem uma cuidadosa consideração
da propagação de erros e da estabilidade dos métodos numéricos, bem como a
adoção de técnicas para minimizar o impacto dos erros na precisão e
confiabilidade dos resultados computacionais.

## $ \S 4 $ Adição no sistema de ponto flutuante

Nas próximas seções vamos explicar através de exemplos como são calculadas
a soma, diferença e produto de dois números no sistema de ponto flutuante.

__Exemplo 4.1:__ Calcule a soma dos números $ 3.15 \times 10^2 $ e $ 1.26 \times 10^1
$, utilizando arredondamento com desempate para cima.

_Solução:_ Antes de realizar a adição, precisamos garantir que os expoentes sejam
iguais. Para isto, vamos "deslocar o ponto" do número com o _menor_ expoente
para que passe a ter o mesmo expoente que o outro número:
\begin{equation*}
    1.26 \times 10^1 = 0.126 \times 10^2 \quad \text{(igualando os expoentes)}
\end{equation*}
Agora podemos realizar a adição:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 3&.15 & \times & 10^2 & \\
        &+& 0&.126 & \times & 10^2 & \text{(adicionando)} \\
        &{=}& 3&.276 & \times & 10^2 & \text{(arredondando)} \\
        &{\approx} & 3&.28 & \times & 10^2 & \text{(resultado)}
    \end{aligned}
\end{equation*}
Note que a mantissa do resultado intermediário $ 3.276 \times 10^2 $ foi
representada com precisão adicional antes de ser convertida por arredondamento a
um resultado final que segue nossas convenções.

__Exemplo 4.2:__ Calcule a soma dos números $ 6.21 \times 10^{2} $ e
$ 5.73 \times 10^{0} $, utilizando truncamento se necessário.

_Solução:_ Mais uma vez, precisamos igualar os expoentes antes de
realizar a adição. Para isto ajustamos o número com o _menor_ expoente:
\begin{equation*}
    5.73 \times 10^0 = 0.0573 \times 10^2 \quad \text{(igualando os expoentes)}
\end{equation*}
Calculando a soma:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 6&.21 & \times & 10^2 \\
        &+& 0&.0573 & \times & 10^2 & \text{(adicionando)}\\
        &{=}& 6&.2673 & \times & 10^2 & \text{(truncamento)} \\
        &{\approx} & 6&.26 & \times & 10^2 & \text{(resultado)}
    \end{aligned}
\end{equation*}
Observe o uso do truncamento no último passo para trazer a mantissa do resultado
intermediário de volta ao nosso formato.

__Exemplo 4.3__: Adicione $ 9.53 \times 10^{-1} $ a $ 7.20 \times 10^{-2} $ usando
arredondamento, com desempate por dígito menos significativo par se necessário.

_Solução:_ Igualando o expoente do segundo número ao do primeiro:
\begin{equation*}
    7.20 \times 10^{-2} = 0.720 \times 10^{-1} \quad \text{(igualando os expoentes)}
\end{equation*}
Realizando a adição:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 9&.53 &\times& 10^{-1} \\
        &+& 0&.720 &\times& 10^{-1} & \text{(adicionando)}\\
        &{=}& 10&.25 &\times& 10^{-1} & \text{(renormalizando)} \\
        &{=}& 1&.025 &\times& 10^{0} &  \text{(arredondando)}\\
        &{\approx}& 1&.02 &\times& 10^{0} & \text{(resultado)} \\
    \end{aligned}
\end{equation*}
No último passo o resultado do arredondamento foi $ 1.02 \times 10^0 $ e não
$ 1.03 \times 10^0 $ já que, conforme especificado no enunciado, é o primeiro
destes cujo dígito menos significativo é par.

__Exercício:__ Calcule as somas dos números de ponto flutuante abaixo. Suas
respostas devem seguir as convenções enunciadas na $ \S 4.2 $.

(a) $ 2.34 \times 10^2 + 3.87 \times 10^2 $; utilize arredondamento com desempate para cima.

(b) $ 4.23 \times 10^{-4} + 1.38 \times 10^{-3} $; utilize truncamento.

(c) $ 3.24 \times 10^5 + 9.78 \times 10^3 $; utilize arredondamento com desempate por dígito par.

(d) $ 7.36 \times 10^{0} + 4.29 \times 10^{-5} $; utilize arredondamento com desempate para cima.

(e) $ 5.99 \times 10^9 + 8.43 \times 10^{-1} $; utilize truncamento.

(f) $ 1.03 \times 10^9 + 8.97 \times 10^{9} $; utilize arredondamento com desempate por dígito par.

__Exercício (por que o zero é definido por $ 0.00 \times 10^{-9} $?):__

(a) Sejam $ x $ e $ y $ dois números de ponto flutuante com sinal $ + $. Mostre que definindo $ x < y $ se e somente
se
* $ \text{expoente}(x) < \text{expoente}(y) $; ou
* $ \text{expoente}(x) = \text{expoente}(y) $ e $ \text{mantissa}(x) < \text{mantissa}(y) $;

obtemos uma ordenação compatível com a ordenação usual de $ x $ e $ y $ como números reais.

(b) Mostre que esta comparação falharia caso representássemos o zero por $ +0.00 \times 10^{e} $ para qualquer
expoente $ e \ne -9 $.

__Exercício (por que o zero é definido por $ 0.00 \times 10^{-9} $?):__
Suponha (como de fato tínhamos convencionado) que resultados intermediários de
operações aritméticas sejam armazenados com no máximo $ 6 $ dígitos, quaisquer
dígitos excedentes sendo imediatamente descartados.

(a) Mostre que neste caso $ 0.00 \times 10^0 + 9.87 \times 10^{-7} = 0.00 \times 10^{0} $.
Ou seja, se permitíssemos a representação do zero por $ 0.00 \times 10^0 $, teríamos
$ x + 0 = 0 $ para alguns $ x \ne 0 $.

(b) Mais geralmente, mostre que se $ e - d > 3 $, se $ x $ é um número de expoente $ d $,
e se zero fosse representado por $ 0.00 \times 10^e $, então poderíamos ter $
0 + x \ne x $.


__Exercício:__ Explique o resultado na célula de código abaixo.

In [18]:
x = 0.1
y = 0.2
print(x + y)

0.30000000000000004


## $ \S 5 $ Subtração no sistema de ponto flutuante

__Exemplo 5.1__: Calcule a diferença $ 1.52 \times 10^3 - 7.38 \times 10^2 $.

_Solução:_ Como para a adição, inicialmente é necessário deslocar o ponto do
número de menor expoente para que este se iguale ao do outro número:
\begin{equation*}
    7.38 \times 10^{2} = 0.738 \times 10^{3} \quad \text{(igualando os expoentes)}
\end{equation*}
Calculando a diferença:
\begin{equation*}
    \begin{aligned}
        &\phantom{-}& 1&.52 &\times& 10^{3} \\
        &-& 0&.738 &\times& 10^{3} & \text{(subtraindo)}\\
        &{=}& 0&.782 &\times& 10^{3} & \text{(renormalizando)}\\
        &{=}& 7&.82 &\times& 10^{2} & \text{(resultado)} \\
    \end{aligned}
\end{equation*}


__Exemplo 5.2__: Calcule a diferença entre $ 6.42 \times 10^{6} $ e
$ 4.21 \times 10^{4} $ utilizando truncamento.

_Solução:_ Igualando o expoente do segundo número ao primeiro:
\begin{equation*}
    4.21 \times 10^4 = 0.0421 \times 10^6 \quad \text{(igualando os expoentes)}
\end{equation*}
Realizando a subtração:
\begin{equation*}
    \begin{aligned}
        &\phantom{+}& 6&.42 &\times& 10^{6} \\
        &-& 0&.0421 &\times& 10^{6} & \text{(subtraindo)}\\
        &{=}& 6&.3779 &\times& 10^{6} &  \text{(truncando)}\\
        &{\approx}& 6&.37 &\times& 10^{6} &  \text{(resultado)}\\
    \end{aligned}
\end{equation*}

__Exemplo 5.3__: Subtraia $ 9.76 \times 10^{-3} $ de $ 1.95 \times 10^{-4} $
utilizando arrendamento, com desempate por dígito menos significativo par.
_Solução:_
Como o número $ a = 1.95 \times 10^{-4} $ do qual está sendo feita a subtração
(_minuendo_) é menor que o número $ b = 9.76 \times 10^{-3} $ que será subtraído
(_subtraendo_), é mais fácil calcular $ b - a $ e depois observar que
$ a - b = -(b - a) $.
Como antes, precisamos primeiro igualar os expoentes:
\begin{equation*}
    1.95 \times 10^{-4} = 0.195 \times 10^{-3} \quad \text{(igualando os expoentes)}
\end{equation*}
Então
\begin{equation*}
    \begin{aligned}
        &\phantom{-}& 9&.76 &\times& 10^{-3} \\
        &{-}& 0&.195 &\times& 10^{-3} & \text{(subtraindo)} \\
        &{=}& 9&.565 &\times& 10^{-3} & \text{(arredondando)}  \\
        &{\approx}& 9&.56 &\times& 10^{-3} & \text{(resultado)} 
    \end{aligned}
\end{equation*}
Assim, a diferença de $ 1.95 \times 10^{-4} $ e $ 9.76 \times 10^{-3} $ usando o
modo de aproximação indicado é $ -9.56 \times 10^{-3} $.

__Exercício:__ Calcule as diferenças abaixo. Suas respostas devem seguir a
convenção acima, ou seja, os expoentes devem estar entre $ 9 $ e $ -9 $ e as
mantissas devem possuir exatamente três dígitos significativos.

(a) $ 6.71 \times 10^1 - 3.42 \times 10^0 $ utilizando truncamento.

(b) $ 9.12 \times 10^{-3} - 2.46 \times 10^{-2} $ utilizando arredondamento com desempate para cima.

(c) $ 1.98 \times 10^6 - 3.24 \times 10^4 $ utilizando arredondamento com desempate por dígito menos significativo par.

(d) $ 7.25 \times 10^{-7} - 5.93 \times 10^{-5} $ utilizando truncamento.

(e) $ 4.73 \times 10^8 - 3.92 \times 10^8 $ utilizando arredondamento com desempate por dígito menos significativo par.

(f) $ 1.01 \times 10^{-9} - 1.32 \times 10^{-9} $ utilizando arredondamento com desempate para cima.

__Exercício:__ Utilizando arredondamento com desempate para cima, mostre que no nosso sistema:

(a) Se $ \vert \delta \vert < \frac{1}{2} \times 10^{-3} $, então
$ 1 \pm \delta = 1 $.

(b) Se $ \varepsilon^2 < \frac{1}{2} \times 10^{-3} $,
então
$$
1 - (1 - \varepsilon)(1 + \varepsilon) = 0\,.
$$

## $ \S 6 $ Multiplicação no sistema de ponto flutuante

__Exemplo 6.1:__ Calcule o produto entre $ 5.23 \times 10^2 $ e $ 1.06 \times 10^{-1} $
utilizando arredondamento, com desempate por par.

_Solução:_ Primeiramente multiplicamos as mantissas e somamos os expoentes, depois
renormalizamos e aproximamos o resultado conforme a necessidade:
\begin{equation*}
\begin{aligned}
&\phantom{{=}}& 5&.23 &\times& 10^2 & \\
&\times& 1&.06 &\times& 10^{-1} \quad & \text{(mutiplicando mantissas)} \\
\hline
&{=} &  (5&.23 & & \quad & \\
&+ & &.000 & & & \\
&+ & &.318) & \times & 10^{2 - 1} \quad & \text{(adicionando expoentes)} \\
&{=}& 5&.5438 & \times & 10^{1} \quad & \text{(arredondando)} \\
&{\approx} & 5&.54 & \times & 10^{1} \quad & \text{(resultado)} \\
\end{aligned}
\end{equation*}

__Exemplo 6.2:__ Multiplique $ 3.71 \times 10^3 $ por $ 4.29 \times 10^{-2} $
utilizando arredondamento com desempate por cima.

__Solução:__ 
\begin{equation*}
\begin{aligned}
&\phantom{=}& 3&.71 &\times& 10^3 & \\
&\times& 4&.29 &\times& 10^{-2} \quad & \text{(mutiplicando mantissas)} \\
\hline
&{=} & ( 14&.84& & \quad & \\
&+ & &.742 & & & \\
&+ & &.3339) & \times & 10^{3 - 2} \quad & \text{(adicionando expoentes)} \\
&{=}& 15&.9159 & \times & 10^{1} \quad & \text{(renormalizando)} \\
&{=}& 1&.59159 & \times & 10^{2} \quad & \text{(arredondando)} \\
&{\approx} & 1&.59 & \times & 10^{2} \quad & \text{(resultado)} \\
\end{aligned}
\end{equation*}


__Exemplo 6.3:__ Calcule o produto de $ 2.13 \times 10^{-5} $ e 
$ \times 4.81 \times 10^{-5} $ utilizando truncamento se necessário.

__Solução:__ 
\begin{equation*}
\begin{aligned}
&\phantom{=}& 2&.13 &\times& 10^{-5} & \\
&\times& 4&.81 &\times& 10^{-5} \quad & \text{(mutiplicando mantissas)} \\
\hline
&{=} & ( 8&.52& & \quad & \\
&{+} & 1&.704 & & & \\
&{+} & &0.213) & \times & 10^{-5-5} \quad & \text{(adicionando expoentes)} \\
&{=}& 10&.2453 & \times & 10^{-10} \quad & \text{(renormalizando)} \\
&{=}& 1&.2453 & \times & 10^{-9} \quad & \text{(truncando)} \\
&{\approx} & 1&.24 & \times & 10^{-9} \quad & \text{(resultado)} \\
\end{aligned}
\end{equation*}

__Exercício:__ Calcule os produtos dos números abaixo no nosso sistema de ponto flutuante.

(a) $ 6.71 \times 10^1 $ e $ 3.42 \times 10^0 $, utilizando truncamento.

(b) $ 9.12 \times 10^{3} $ e $ -2.40 \times 10^{-2} $, utilizando arredondamento com desempate para cima.

(c) $ -1.98 \times 10^6 $ e $ 3.24 \times 10^4 $, utilizando arredondamento com desempate por dígito menos significativo par.

(d) $ 7.25 \times 10^{-2} $ e $ 5.93 \times 10^{4} $, utilizando truncamento.

(e) $ 4.73 \times 10^3 $ e $ -3.92 \times 10^5 $, utilizando arredondamento com desempate por dígito par.

(f) $ 5.01 \times 10^{-3} $ e $ 1.32 \times 10^{-7} $, utilizando arredondamento com desempate para cima.

__Exercício__: 

(a) Seja $ a = 2.50 \times 10^4 $. Qual é o maior número $ x $ no nosso sistema
    de ponto flutuante tal que o produto $ ax $ _não_ resulta num erro de
    overflow?

(b) Seja $ b = 2.50 \times 10^{-3} $. Qual é o menor número $ y $ no nosso sistema
    de ponto flutuante tal que o produto $ by $ _não_ resulta num erro de
    underflow?

📝 A divisão na aritmética de ponto flutuante é uma operação mais complexa
que a adição, subtração ou multiplicação, por isto não será considerada
detalhadamente aqui. Em resumo, ela consiste dos três passos:

* Extrair as mantissas e os expoentes.
* Calcular a diferença entre os expoentes do divisor e do dividendo.
* Calcular o quociente das mantissas, por exemplo usando divisão longa, método
  de Newton ou método de Goldschmidt. Como antes, este quociente pode precisar
  ser ajustado (aproximado ou renormalizado) para que siga o formato de ponto
  flutuante especificado.